In [12]:
import numpy as np
import pandas as pd
import itertools
import cPickle
# import datatime
import hashlib
import locale    # 多语言模块
import pycountry
import scipy.io as sio
import scipy.sparse as ss
import scipy.spatial.distance as ssd

from collections import defaultdict
from sklearn.preprocessing import normalize

In [13]:
train_data = pd.read_csv('data/train.csv')
store_data = pd.read_csv('data/store.csv')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
Store            1017209 non-null int64
DayOfWeek        1017209 non-null int64
Date             1017209 non-null object
Sales            1017209 non-null int64
Customers        1017209 non-null int64
Open             1017209 non-null int64
Promo            1017209 non-null int64
StateHoliday     1017209 non-null object
SchoolHoliday    1017209 non-null int64
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


In [6]:
store_data.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


1.清洗数据类

In [ ]:
class DataCleaner:
    
    def __init__(self):
        # 载入 locales
        self.localeIdMap = defaultdict(int)
        for i,l in enumerate(locale.locale_alias.keys()):
            self.localeIdMap[l] = i + 1
        # 载入countries
        self.countryIdMap = defaultdict(int)
        ctryIdx = defaultdict(int)
        for i,c in enumerate(pycountry.countries):
            self.countryIdMap[c.name.lower()] = i + 1
            if c.name.lower() == 'usa':
                ctryIdx["US"] = i
            if c.name.lower() == 'canada':
                ctryIdx["CA"] = i
        for cc in ctryIdx.keys():
            for s in pycountry.subdivisions.get(country_code==cc):
                self.countryIdMap[s.name.lower()] = ctryIdx[cc] + 1
        # 载入gender id字典
        self.genderIdMap = defaultdict(int,{"male":1,"female":2})
    
    def getLocaleId(self,locstr):
        return self.localeIdMap[locstr.lower()]
    
    def getGenderId(self,genderstr):
        return self.genderIdMap[genderstr]
    
    def getJoinedYearMonth(self,dateString):
        dttm = datetime.datetime.strptime(dateString,"%Y-%m-%dT%H:%M:%S.%fZ")
        return "".join(str(dttm.year),str(dttm.month))
    
    def getCountryId(self,location):
        if(isinstance(location,str)) and len(location.strip())>0 and location.rfind("  "):
            return self.countryIdMap[location[location.rindex("  ") + 2:].lower()]
        else:
            return 0
    
    def getBirthYearInt(self,birthYear):
        try:
            return 0 if birthYear == "None" else int(birthYear)
        except:
            return 0
    
    def getTimezoneInt(self,timezone):
        try:
            return int(timezone)
        except:
            return 0
    
    def getFeatureHash(self,value):
        if len(value.strip()) == 0:
            return -1
        else:
            return int(hashlib.sha224(value).hexdigest()[0:4],16)
    
    def getFloatValue(self,value):
        if len(value.strip()) == 0:
            return 0.0
        else:
            return float(value)